In [46]:
# importing libraries
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np
import nltk
# nltk.download('all') #(Download the package if not downloaded already - Large file) #uncomment it out

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [47]:
#crawling using beautiful soup

url="https://in.mashable.com/"
r=requests.get(url)
soup=BeautifulSoup(r.content,'html5lib') 
table=soup.findAll('a',attrs={'class':'title'})
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://in.mashable.com'+row['href'])

df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

    
dataset=pd.DataFrame(df)
dataset.head(10)

,Title,Text,Summary,Keywords
0,'Anwar Ka Ajab Kissa' Review: Nawazuddin Siddi...,"Anwar Ka Ajab Kissa or Sniffer, written and di...","Anwar Ka Ajab Kissa or Sniffer, written and di...","[kissa, sound, movie, watchable, tries, anwar,..."
1,COVID-19: Oxford Vaccine To Be Available In In...,And after months of waiting while Coronavirus ...,Pardon my philosophy but the pandemic isn’t ov...,"[rs, oxford, poonawalla, priced, results, 2021..."
2,Facebook Messenger Patched A Bug That Would Al...,One of the most widely-known spying methods is...,"Fret not, the bug has been patched though, cla...","[researchers, allow, calls, program, messenger..."
3,Amazon's Making An Upgraded Version Of Its Ech...,Amazon announced today that its experimental s...,"Echo Frames have now exited its testing phase,...","[volume, frames, version, amazons, upgraded, m..."
4,NVIDIA GeForce NOW Streaming Service Comes To ...,NVIDIA is using a web-based client to bypass A...,NVIDIA is using a web-based client to bypass A...,"[game, argued, nvidia, google, comes, streamin..."
5,Microsoft Teams Now Offers 24-Hour Video Calls...,"Just in time for the holiday season, Microsoft...","Just in time for the holiday season, Microsoft...","[free, teams, calls, limit, season, offers, zo..."
6,Netflix For Free In India? All You Need To Kno...,Nothing grabs eyeballs like the word FREE does...,Netflix knows this and it's planning to do som...,"[free, sign, access, streaming, netflix, wont,..."
7,‘Godmothered’ Trailer: Jillian Bell Gives Isla...,‘Tis the season to be jolly and it’s raining h...,"In fact, if you’ve watched and liked Enchanted...","[godmothered, eleanor, isla, jillian, trailer,..."
8,'Middle Class Melodies' Review: A Mediocre Mov...,"Middle Class Melodies, directed by Vinod Anant...","Now, Middle Class Melodies tries to encapsulat...","[class, deverakonda, best, melodies, isnt, med..."
9,Akshay Kumar Serves Rs 500 Crore Defamation No...,The Sushant Singh Rajput death case opened a c...,The Sushant Singh Rajput death case opened a c...,"[videos, sushant, rs, kumar, singh, client, no..."


In [48]:
FILEPATH="OnlineNewsPopularity.csv" #dataset from UCI Repo
data1=pd.read_csv(FILEPATH)
data1.head(10)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.6,-0.195000,-0.400,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.0,-0.224479,-0.500,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.0,-0.242778,-0.500,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.8,-0.125000,-0.125,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.5,-0.238095,-0.500,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [49]:
clean_cols={x: x.lower().strip() for x in list(data1)}
data1=data1.rename(index=str,columns=clean_cols)

In [50]:
train_set, test_set = train_test_split(data1, test_size=0.4, random_state=42)

drop_attributes_list = ['url','shares', 'timedelta','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess']

x_train = train_set.drop(drop_attributes_list, axis=1)
y_train = train_set['shares']

x_test = test_set.drop(drop_attributes_list, axis=1)
y_test = test_set['shares']



In [51]:
print("Training classifier...")
clf=RandomForestRegressor(n_estimators=10, random_state=42)
clf.fit(x_train,y_train)
print("Trained successful")

Training classifier...
Trained successful


In [52]:

result=pd.DataFrame(clf.predict(x_train),list(y_train))
result.reset_index(level=0, inplace=True)
result_df = result.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
result_df.head(10)

,Actual shares,Predicted shares
0,1900,2200.0
1,1800,1680.0
2,790,841.0
3,872,884.0
4,2500,2383.6
5,860,1750.0
6,1000,1047.8
7,47300,25569.6
8,20400,16584.5
9,2500,2470.0


In [53]:
FILEPATH2="abcnews-date-text.csv"
data = pd.read_csv(FILEPATH2, error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
print("Number of headlines in the dataset:", len(documents))
# print(len(documents))
# print(documents[:5])
documents.head(10)

Number of headlines in the dataset: 1186018


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
5,ambitious olsson wins triple jump,5
6,antic delighted with record breaking barca,6
7,aussie qualifier stosur wastes four memphis match,7
8,aust addresses un security council over iraq,8
9,australia is locked into war timetable opp,9
